# ScanPy Integrated Analysis

In ScanPy there are two ways to do this:
 * BBKNN
 * MNN_correct
   * Requires mnnpy (installed via pip, not conda due to missing)
 
 https://icb-scanpy.readthedocs-hosted.com/en/latest/api/scanpy.external.html

In [11]:
import numpy as np
import pandas as pd
import scanpy as sc
import scanpy.external as sce

We need to have the same number of features.

In [4]:
# Failed: Too much unfiltered data to process.
#
#ad_ctr = sc.read_10x_mtx(
#    './sources/ctr',  # the directory with the `.mtx` file
#    var_names='gene_symbols',                  # use gene symbols for the variable names (variables-axis index)
#    cache=True)
#
#ad_ctr.var_names_make_unique()  # this is unnecessary if using 'gene_ids'
#
#ad_sin = sc.read_10x_mtx(
#    './sources/siN6amt1',  # the directory with the `.mtx` file
#    var_names='gene_symbols',                  # use gene symbols for the variable names (variables-axis index)
#    cache=True)
#
#ad_sin.var_names_make_unique()  # this is unnecessary if using 'gene_ids'

In [5]:
# Failed: Need the same number of variables
#
ad_ctr = sc.read("checkpoints/scanpy.ctr.cc.h5ad")
ad_sin = sc.read("checkpoints/scanpy.sin.cc.h5ad")

print("Ctrl:","%d x %d" % (ad_ctr.n_vars, ad_ctr.n_obs))
print("SIN :","%d x %d" % (ad_sin.n_vars, ad_sin.n_obs))

Only considering the two last: ['.cc', '.h5ad'].
Only considering the two last: ['.cc', '.h5ad'].
Only considering the two last: ['.cc', '.h5ad'].
Only considering the two last: ['.cc', '.h5ad'].
Ctrl: 1721 x 3325
SIN : 1674 x 4416


In [6]:
# Combine on features
ad_combi = ad_ctr.concatenate(ad_sin, join="outer", batch_key="SIN")
print("combi:","%d x %d" % (ad_combi.n_vars, ad_combi.n_obs))

combi: 2566 x 7741


2566 features using an outer join. Let's seperate them on OUTER and see if we eliminate the batch effect

Try it on inner join if that fails.

In [24]:
# Then split them out
#tmp = 
ad_ctr = ad_combi[ad_combi.obs["SIN"].astype("int32") == 0]
ad_sin = ad_combi[ad_combi.obs["SIN"].astype("int32") == 1]

In [25]:
print("Ctrl:","%d x %d" % (ad_ctr.n_vars, ad_ctr.n_obs))
print("SIN :","%d x %d" % (ad_sin.n_vars, ad_sin.n_obs))


Ctrl: 2566 x 3325
SIN : 2566 x 4416


Okay we now have two sets with the same number of features.

In [27]:
#mnn_data = sce.pp.mnn_correct(ad_ctr,ad_sin)

Failed possibly because the number of cells are also different....

Let's try to filter the low quality cells from the SIN matrix to bring it down to 3325

In [46]:
sum(ad_sin.obs["n_counts"] > 29200)

3325

Okay so if we set a filter for > 30000 we can get the SIN matrix down to 3325.

In [47]:
ad_sin = ad_sin[ad_sin.obs["n_counts"] > 29200]
print("Ctrl:","%d x %d" % (ad_ctr.n_vars, ad_ctr.n_obs))
print("SIN :","%d x %d" % (ad_sin.n_vars, ad_sin.n_obs))

Ctrl: 2566 x 3325
SIN : 2566 x 3325


Okay, now they have identical dimentions. Let's try the mnn again

In [49]:
#mnn_data = sce.pp.mnn_correct(ad_ctr,ad_sin)

ValueError: not enough values to unpack (expected 3, got 1)

This also fails because there's just too many to unpack. According to [this comment](https://github.com/theislab/scanpy/issues/449) we should only use a set of highly variable genes.

So let's try to merge only on the set of highly variable genes in each batch.

In [7]:
ad_ctr = ad_combi[ad_combi.obs["SIN"].astype("int32") == 0]
ad_sin = ad_combi[ad_combi.obs["SIN"].astype("int32") == 1]
print("Ctrl:","%d x %d" % (ad_ctr.n_vars, ad_ctr.n_obs))
print("SIN :","%d x %d" % (ad_sin.n_vars, ad_sin.n_obs))

Ctrl: 2566 x 3325
SIN : 2566 x 4416


In [8]:
ctrgenes_highly = ad_ctr.var[ad_ctr.var["highly_variable-0"]==True].index
ctrgenes_highly = [x for x in ctrgenes_highly.values.astype("str")]

singenes_highly = ad_sin.var[ad_sin.var["highly_variable-1"]==True].index
singenes_highly = [x for x in singenes_highly.values.astype("str")]

print("Highly Expressed genes:", "ctr", len(ctrgenes_highly), ", sin", len(singenes_highly))

Highly Expressed genes: ctr 1721 , sin 1674


In [9]:
mutual_genes = [x for x in singenes_highly if x in ctrgenes_highly]
len(mutual_genes)

829

829 mutual highly expressed genes.

In [12]:
mnn_data = sce.pp.mnn_correct(ad_ctr,ad_sin, var_subset=mutual_genes)

Performing cosine normalization...
Starting MNN correct iteration. Reference batch: 0
Step 1 of 1: processing batch 1
  Looking for MNNs...
  Computing correction vectors...
  Adjusting variance...


ZeroDivisionError: float division

ZeroDivisionError: float division

ZeroDivisionError: float division

ZeroDivisionError: float division

...

ZeroDivisionError: float division

Exception ignored in: 'mnnpy._utils._adjust_s_variance'
ZeroDivisionError: float division


ZeroDivisionError: float division

Exception ignored in: 'mnnpy._utils._adjust_s_variance'
ZeroDivisionError: float division


  Applying correction...
MNN correction complete. Gathering output...
Packing AnnData object...
Done.


In [4]:
mnn_data[0].write("checkpoints/scanpy.mnn.h5ad")

NameError: name 'mnn_data' is not defined

In [3]:
import numpy as np
import pandas as pd
import scanpy as sc

ad_integ = sc.read("checkpoints/scanpy.mnn.h5ad")
ad_integ

OSError: Unable to open file (unable to open file: name = 'checkpoints/scanpy.mnn.h5ad', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [83]:
np.nan_to_num(ad_integ)
#sc.tl.pca(ad_integ, svd_solver='arpack')

MemoryError: 

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')